In [ ]:
%run ./modules/load_data_module.ipynb
%run ./modules/extract_info_clean_data_module.ipynb
%run ./modules/update_dataframe_module.ipynb
%run ./modules/sentiment_module.ipynb

In [ ]:
import pandas as pd
import spacy
import string
import regex as re
import nltk
import ast
import copy
import glob
import advertools as adv
import plotly.graph_objects as go


pd.options.display.max_colwidth = 285



from textblob import TextBlob

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


from spacymoji import Emoji
from nltk.corpus import stopwords
from urllib.parse import urlparse
from textblob import Word
from sklearn.metrics.pairwise import cosine_similarity



nlp = spacy.load("en_core_web_sm")
emoji = Emoji(nlp)
nlp.add_pipe("emoji", first=True)


stemmer = nltk.SnowballStemmer("english")


# nltk.download('wordnet')
# nltk.download('omw-1.4')


In [ ]:
tweets_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*tweets.csv')
users_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*users.csv')
replies_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*replies.csv')
quotes_path = glob.glob('/home/niloofar/Developement/Climate-Change/data/data/*quotes.csv')

In [ ]:
user_dtype = {
    "id": "Int64", 
    "username": "string", 
    "created_at": "string", 
    "name": "string", 
    "protected": "boolean", 
    "verified": "boolean", 
    "followers_count": "Int64", 
    "following_count": "Int64", 
    "tweet_count": "Int64", 
    "listed_count": "Int64", 
    "url": "string", 
    "description": "string"
}


tweet_dtype = {
    "id": "Int64",
    "author_id": "Int64",
    "conversation_id": "Int64",
    "created_at": "string",
    "lang": "string",
    "retweet_count": "Int64",
    "reply_count": "Int64",
    "like_count": "Int64",
    "quote_count": "Int64",
    "reply_settings": "string",
    "text": "string"
}


conversation_dtype = {
    "id": "Int64",
    "author_id": "Int64",
    "conversation_id": "Int64",
    "created_at": "string",
    "lang": "string",
    "retweet_count": "Int64",
    "reply_count": "Int64",
    "like_count": "Int64",
    "quote_count": "Int64",
    "referenced_tweets": "string",
    "text": "string"
}


In [ ]:
load_tweets = get_df(tweets_path, tweet_dtype)
load_replies = get_df(replies_path, conversation_dtype)
load_quotes = get_df(quotes_path, conversation_dtype).dropna()
load_users = get_df(users_path, user_dtype)

In [ ]:
users = clean_users(load_users)

# initial dataframe cleaning

In [ ]:
# tweet = copy.deepcopy(tweets)
# reply = copy.deepcopy(replies)
# quote = copy.deepcopy(quotes)

In [ ]:
sample_tweets = pd.read_csv('/home/niloofar/Developement/Climate-Change/data/sample_data/sample_tweets.csv')
sample_replies = pd.read_csv('/home/niloofar/Developement/Climate-Change/data/sample_data/sample_replies.csv')
sample_quotes = pd.read_csv('/home/niloofar/Developement/Climate-Change/data/sample_data/sample_quotes.csv')

# GEN-DATA 

In [ ]:
sample_tweets['hashtags'] = add_hashtags(sample_tweets)

In [ ]:
sample_tweets['hashtag_count'] = add_nbr_hashtags(sample_tweets)

In [ ]:
sample_tweets['uppercase_count'] = add_uppercase_count(sample_tweets)

In [ ]:
sample_tweets['uppercase_pct'] = add_upper_case_pct(sample_tweets)

In [ ]:
sample_tweets['exclamation_mark_count'] = add_exclamation_mark_count(sample_tweets)

In [ ]:
sample_tweets['question_mark_count'] =  add_question_mark_count(sample_tweets)

In [ ]:
sample_tweets['url_count'] = add_url_count(sample_tweets)

In [ ]:
sample_tweets['mention_count'] = add_mention_count(sample_tweets)

In [ ]:
sample_tweets['emojie_count'] = add_emoji_count(sample_tweets)

In [ ]:
sample_tweets['followers_count'] = add_followers_count(sample_tweets, users)

In [ ]:
sample_tweets['engagement_score'] = add_engagement_score(sample_tweets)

In [ ]:
sample_tweets['sentiment_score'] = add_sentiment_score(sample_tweets)

In [ ]:
sample_replies['sentiment_score'] = add_sentiment_score(sample_replies)

In [ ]:
sample_replies.head(1)

In [ ]:
sample_quotes['sentiment_score'] = add_sentiment_score(sample_quotes)

In [ ]:
sample_quotes.head(1)